In [1]:
import os
os.chdir('../../')

In [2]:
import pandas as pd
import pandas_gbq
import re
from google.oauth2 import service_account

In [3]:
SERVICE_ACCOUNT = 'private/bigquery-write-access.json'
DATASET = 'student_counts'
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT)

# UNESCO Data

## Countries Map

In [4]:
path = 'data/unesco_unimelb_countries_map.csv'
index_col = 'unesco_country'
unesco_countries_map_df = pd.read_csv(path)
#unesco_countries_map_df[index_col] = unesco_countries_map_df[index_col].apply(lambda s: re.sub(r"^(the\s)(.*)", r"\2", s))
#unesco_countries_map_df.to_csv(path)

In [5]:
table_full_name = '{}.{}'.format(DATASET, 'unesco_unimelb_countries_map')
unesco_countries_map_df.to_gbq(table_full_name, credentials.project_id, credentials=credentials, if_exists='replace')

1it [00:05,  5.95s/it]


In [6]:
unesco_countries_map_df.set_index(index_col, inplace=True)
unesco_countries_map_df.head()

,unimelb_country
unesco_country,
Afghanistan,Afghanistan
Albania,Albania
Algeria,Algeria
Andorra,Andorra
Angola,Angola


## Counts data

In [7]:
s = 'ASIA: Students from China, both sexes (number)'
r = re.search('.*Students\sfrom\s(the\s)?(.*),\sboth.*', s)
print(r.groups()[1])

China


In [8]:
df = pd.read_csv('data/unesco_students_counts.csv')

df['source_country_original'] = df.Indicator.apply(lambda s: re.search('.*Students\sfrom\s(the\s)?(.*),\sboth.*', s).groups()[1])
df['students_count'] = df.Value.astype(int)

del df['Indicator']
del df['Value']


In [9]:
df = df.join(unesco_countries_map_df, on='source_country_original')
df.rename(columns={"unimelb_country": "source_country"}, inplace=True)
df = df.join(unesco_countries_map_df, on='destination_country_original')
df.rename(columns={"unimelb_country": "destination_country"}, inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121328 entries, 0 to 121327
Data columns (total 11 columns):
EDULIT_IND                      121328 non-null int64
destination_country_code        121328 non-null object
destination_country_original    121328 non-null object
year                            121328 non-null int64
flad_codes                      3996 non-null object
flags                           3996 non-null object
is_imputed                      121328 non-null bool
source_country_original         121328 non-null object
students_count                  121328 non-null int64
source_country                  121328 non-null object
destination_country             121328 non-null object
dtypes: bool(1), int64(3), object(7)
memory usage: 9.4+ MB


In [11]:
df.head()

,EDULIT_IND,destination_country_code,destination_country_original,year,flad_codes,flags,is_imputed,source_country_original,students_count,source_country,destination_country
0,26528,USA,United States of America,2016,NaN,NaN,False,China,309837,China,United States of America
1,26528,USA,United States of America,2015,NaN,NaN,False,China,291063,China,United States of America
2,26528,USA,United States of America,2014,NaN,NaN,False,China,260914,China,United States of America
3,26528,USA,United States of America,2013,NaN,NaN,False,China,225474,China,United States of America
4,26528,USA,United States of America,2012,NaN,NaN,False,China,210452,China,United States of America


In [12]:
table_full_name = '{}.{}'.format(DATASET, 'unesco_students_counts')
df.to_gbq(table_full_name, credentials.project_id, credentials=credentials, if_exists='replace')

1it [00:00,  1.34it/s]


# Country Specific Data

## Countries Map

In [13]:
df = pd.read_csv('data/countries_map.csv')
df.head()

,country,region,uk_name,us_name
0,Afghanistan,Southern and Central Asia,Afghanistan,Afghanistan
1,Albania,Southern and Eastern Europe,Albania,Albania
2,Algeria,North Africa and the Middle East,Algeria,Algeria
3,Andorra,Southern and Eastern Europe,Andorra,Andorra
4,Angola,Sub-Saharan Africa,Angola,Angola


In [14]:
table_full_name = '{}.{}'.format(DATASET, 'uk_us_countries_map')
df.to_gbq(table_full_name, credentials.project_id, credentials=credentials, if_exists='replace')

1it [00:06,  6.02s/it]


In [15]:
df = pd.read_csv('data/canada_countries_map.csv')
df.head()

,canada_name,country
0,Afghanistan,Afghanistan
1,Albania,Albania
2,Algeria,Algeria
3,Andorra,Andorra
4,Angola,Angola


In [16]:
table_full_name = '{}.{}'.format(DATASET, 'can_countries_map')
df.to_gbq(table_full_name, credentials.project_id, credentials=credentials, if_exists='replace')

1it [00:06,  6.10s/it]


## AUS ennrollments/commencements

In [17]:
study_level_map = {"Diploma, Associate, Advanced Dips": "Other", "UG": "UG", "HDCW": "PG", "RHD": "PG"}

In [18]:
df = pd.read_csv('data/AUS_students_data.csv')
df['study_level'] = df.study_level_he.apply(lambda x: study_level_map[x])
df.head()

,year,country,study_level_original,study_level_he,broad_field_of_education,detailed_field_of_education,enrolments,commencements,study_level
0,2007,Argentina,Bachelor Degree,UG,_Dual Qualification,_Dual Qualification,1,0,UG
1,2007,Argentina,Bachelor Degree,UG,Creative Arts,Audio Visual Studies,1,1,UG
2,2007,Argentina,Bachelor Degree,UG,Creative Arts,"Creative Arts, n.f.d.",1,1,UG
3,2007,Argentina,Bachelor Degree,UG,Creative Arts,Music,1,1,UG
4,2007,Argentina,Bachelor Degree,UG,Creative Arts,Photography,1,1,UG


In [19]:
table_full_name = '{}.{}'.format(DATASET, 'aus_int_student_counts')
df.to_gbq(table_full_name, credentials.project_id, credentials=credentials, if_exists='replace')

1it [00:01,  1.24s/it]



## USA ennrollments

In [20]:
df = pd.read_csv('data/USA_enrollments.csv', index_col=['country', 'year'])
df.columns.name = 'study_level'
df = df.stack()
df.name = 'students_count'
df = df.apply(lambda x: int(x.replace(',', ''))).reset_index()

In [21]:
table_full_name = '{}.{}'.format(DATASET, 'us_int_student_counts')
df.to_gbq(table_full_name, credentials.project_id, credentials=credentials, if_exists='replace')

1it [00:05,  5.37s/it]


## UK ennrollments

In [22]:
study_level_map = {"UG": "UG", "UG Other": "UG", "PG R": "PG", "PG CW": "PG", "PG Other": "PG"}

In [23]:
df = pd.read_csv('data/UK_students_count.csv', index_col=['country', 'year'],
                 encoding="utf-8").fillna(0).astype(int)
df.columns.name = 'detailed_study_level'
df = df.stack()
df.name = 'students_count'
df = df.reset_index()
df['study_level'] = df.detailed_study_level.apply(lambda x: study_level_map[x])
df['country'] = df.country.str.strip()
df.head()

,country,year,detailed_study_level,students_count,study_level
0,Albania,2004/2005,PG R,30,PG
1,Albania,2004/2005,PG CW,55,PG
2,Albania,2004/2005,PG Other,15,PG
3,Albania,2004/2005,UG,115,UG
4,Albania,2004/2005,UG Other,20,UG


In [24]:
table_full_name = '{}.{}'.format(DATASET, 'uk_int_student_counts')
df.to_gbq(table_full_name, credentials.project_id, credentials=credentials, if_exists='replace')

1it [00:05,  5.68s/it]


## Canada

In [25]:
df = pd.read_csv('data/Canada_international_students.csv', encoding="utf-8")
df['students_count'] = df.students_count.fillna(0).astype(int)
df['country'] = df.country.str.strip()
df.head()

,year,institution_type,country,students_count
0,2000/2001,University,Greenland,0
1,2001/2002,University,Greenland,3
2,2002/2003,University,Greenland,0
3,2003/2004,University,Greenland,0
4,2004/2005,University,Greenland,0


In [26]:
table_full_name = '{}.{}'.format(DATASET, 'canada_int_student_counts')
df.to_gbq(table_full_name, credentials.project_id, credentials=credentials, if_exists='replace')

1it [00:06,  6.27s/it]
